In [1]:
import pandas as pd
import gensim
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import spacy
%run calculateSentence.ipynb
import preprocessing
import tfidf
pd.set_option('max_colwidth', 5000)

In [2]:
#Importazione DB

data = pd.read_csv('DB/Preprocessed_ontology.csv')

In [3]:
#Importazione WordEmbeddings

word_2_vec_model = gensim.models.word2vec.Word2Vec.load('DB/word2vec_trained.bin')
word_2_vec_model

In [4]:
def normalize(words):
    words = preprocessing.clear_text(words,"w2v")
    return words

In [5]:
def deleteRow(data):
    
    row = []
                
    for tmp in data:
        if tmp[2] != []:
            row.append(tmp)
    
    return row

In [23]:
def w2v_result(txt, tag):
    stringSearch = normalize(txt)

    #print(stringSearch)

    numberResult = 3

    #Viene vettorizzata la query dell'utente

    vectorSearch = np.array([questionToVec(stringSearch, word_2_vec_model)])
    
    #Carico il file con gli embeddings dei titoli sui quali effettuare la corrispondenza
    allFuncEmb = pd.read_csv('DB/functionEmbeddings.csv').values

    #Calcolo della similarità del coseno for le query e tutti i titoli

    similarityCosine = pd.Series(cosine_similarity(vectorSearch, allFuncEmb)[0])
    
    #Lista che conterrà coppie i,j con i indice e j score di similarità
    results = []
    
    #Se tag è null vuol dire che siamo alla prima iterazione e non dobbiamo filtrare su nessun tag
    if tag == None :   
        #Una volta calcolati gli score di similarità rendo i 10 più alti
        for i,j in similarityCosine.nlargest(numberResult).iteritems():
            #print(i, data.Body[i])
            #Filtro i tag da rendere come risultato, eliminando quelli che sono già presenti nella string search per facilitare prossimi passaggi 
            description = data.iloc[[i]].description.item()
            function = data.iloc[[i]].function.item()
                    
            results.append(tuple([function,description, j]))
        #results = deleteRow(results)
    return results

In [12]:
data.iloc[[5]]

,function,description,return_value,description_processed
5,java.awt.GridBagLayout-minimumLayoutSize(java.awt.Container),Determines the minimum size of the parent container using this grid bag layout. Most applications do not call this method directly.,the minimum size of the parent container,determines minimum size parent container using grid bag layout applications call method directly


In [25]:
w2v_result('for loop in Python',None)

[('java.util.List-iterator()',
  'Returns an iterator over the elements in this list in proper sequence.   ',
  0.31718905633834316),
 ('java.util.ArrayList-iterator()',
  'Returns an iterator over the elements in this list in proper sequence.   The returned iterator is fail-fast.   ',
  0.28686708738494976),
 ('javax.smartcardio.CardTerminals-list()',
  'Returns an unmodifiable list of all available terminals.   ',
  0.252603254153486)]